In [14]:
import csv
import pandas as pd
import word2vec

#Orignal csv file was too big so now using this we can still access the information as a zip file
import zipfile

In [15]:

archive = zipfile.ZipFile('merged_Table.zip', 'r')
merged_table = archive.open('merged_Table.csv')

table = pd.read_csv(merged_table)

In [16]:
table = table.drop(columns="TO BE DELETED")
table = table.drop(columns="TO BE DELETED.1")
table = table.drop(columns="TO BE DELETED.2")
table = table.drop(columns="TO BE DELETED.3")
table = table.drop(columns="TO BE DELETED.4")

In [17]:
table.head()

,Unnamed: 0,Source of ADL,DrugLabel (ADL),DBID,pubchem_id,Source of ADL.1,stitch_id_flat,stitch_id_stereo,umls_cui_from_meddra,method,umls_name,Type_of_term,meddra_id,meddra_name,drug_name,DBID.1,ind_name,ind_id
0,0,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...,DB00014,47725,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,CID100047725,CID000047725,C0014175,NLP_precondition,Endometriosis,LLT,C0014175,Endometriosis,Goserelin,DB00014,Endometriosis,C0014175
1,1,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...,DB00014,47725,prescription/20130626_6bd94a89-f18a-45e5-d0a4-...,CID100047725,CID000047725,C0014175,NLP_precondition,Endometriosis,PT,C0014175,Endometriosis,Goserelin,DB00014,Endometriosis,C0014175
2,2,prescription/20120908_870d88c3-4388-454b-9360-...,INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...,DB00035,27991,prescription/20120908_870d88c3-4388-454b-9360-...,CID100027991,CID005311065,C0019069,NLP_indication,Hemophilia A,LLT,C0019069,Factor VIII deficiency,Desmopressin,DB00035,Hemophilia A,C0019069
3,3,prescription/20120908_870d88c3-4388-454b-9360-...,INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...,DB00035,27991,prescription/20120908_870d88c3-4388-454b-9360-...,CID100027991,CID005311065,C0019069,NLP_indication,Hemophilia A,PT,C0019069,Factor VIII deficiency,Desmopressin,DB00035,Hemophilia A,C0019069
4,4,prescription/20130131_9fd48eec-aa24-4888-b99f-...,INDICATIONS AND USAGE\n\n\n\n\n\n Hemophilia A...,DB00035,27991,prescription/20130131_9fd48eec-aa24-4888-b99f-...,CID100027991,CID005311065,C0019069,NLP_indication,Hemophilia A,LLT,C0019069,Factor VIII deficiency,Desmopressin,DB00035,Hemophilia A,C0019069


Extract the ADL column in order to generate a context dataframe for text mining.

In [18]:
context = table['DrugLabel (ADL)']

In [19]:
context.head()

0    1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...
1    1 INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\n1.1 S...
2    INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...
3    INDICATIONS AND USAGE\n\n\n\n\n\n\n\n\nHemophi...
4    INDICATIONS AND USAGE\n\n\n\n\n\n Hemophilia A...
Name: DrugLabel (ADL), dtype: object

Now that we can see that the text is correctly obtained from the context dataframe, we now transform it into a list such that it is readable

In [20]:
dfToList = context.tolist()


The code from below this point will relate to the word2vec model.

This is the part which imports the gensim package which is primarily used for the word2vec model. It also includes the logging package which is useful for reading and analysing performance

In [21]:
 
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 

This is a test to see if the entry gets printed out as a sentence. We wish for the context to be printed out such that we can see the indications and usage for each instance.

In [23]:
#data_file2 = context.to_string()

for i, line in enumerate(dfToList):
    print(line)
    if i == 1:
        break


#data_file2 = nltk.tokenize()

1 INDICATIONS AND USAGE








1.1 Stage B2-C Prostatic Carcinoma
ZOLADEX is indicated for use in combination with flutamide for the management of locally confined Stage T2b-T4 (Stage B2-C) carcinoma of the prostate. Treatment with ZOLADEX and flutamide should start 8 weeks prior to initiating radiation therapy and continue during radiation therapy [see Dosage and Administration (2.1)and Clinical Studies (14.1)]. 





1.2 Prostatic Carcinoma
ZOLADEX is indicated in the palliative treatment of advanced carcinoma of the prostate [see Dosage and Administration (2.2) and Clinical Studies (14.2)]. 





1.3 Endometriosis
ZOLADEX is indicated for the management of endometriosis, including pain relief and reduction of endometriotic lesions for the duration of therapy. Experience with ZOLADEX for the management of endometriosis has been limited to women 18 years of age and older treated for 6 months [see Dosage and Administration (2.3) andClinical Studies (14.3)]. 





1.4 Endometrial Thinn

YES! This is shows the complete ADL.

This part will now open the list and convert each word in the list into a new, tokenized list called "documents"

In [24]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    
    for i, line in enumerate (input_file): 

        if (i%10000==0):
            logging.info ("read {0} reviews".format (i))
        # do some pre-processing and return a list of words for each review text
        yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (dfToList))
logging.info ("Done reading data file")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

2019-03-25 15:10:37,304 : INFO : read 0 reviews
2019-03-25 15:10:40,305 : INFO : read 10000 reviews
2019-03-25 15:10:44,278 : INFO : read 20000 reviews
2019-03-25 15:10:48,062 : INFO : read 30000 reviews
2019-03-25 15:10:51,897 : INFO : read 40000 reviews
2019-03-25 15:10:54,462 : INFO : Done reading data file


In [28]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-03-25 15:15:18,138 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-03-25 15:15:18,140 : INFO : collecting all words and their counts
2019-03-25 15:15:18,141 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-03-25 15:15:18,467 : INFO : PROGRESS: at sentence #10000, processed 1855924 words, keeping 3592 word types
2019-03-25 15:15:18,969 : INFO : PROGRESS: at sentence #20000, processed 4886712 words, keeping 4703 word types
2019-03-25 15:15:19,500 : INFO : PROGRESS: at sentence #30000, processed 7802421 words, keeping 5667 word types
2019-03-25 15:15:19,914 : INFO : PROGRESS: at sentence #40000, processed 10643351 words, keeping 6489 word types
2019-03-25 15:15:20,194 : INFO : collected 7524 word types from a corpus of 12480701 raw words and 47631 sentences
2019-03-25 15:15:20,195 : INFO : Loading a fresh vocabulary
2019-03-25 15:15:20,229 : INFO : min_count=2 retains 7524 unique words (100% of original 7524, drop

(95608620, 124807010)

Here you can check out the similiarity between word vectors here. It chooses the 10 most similiar words to the one choosen. 

In [32]:
w1 = "surgery"
model.wv.most_similar (positive=w1)

[('hysterectomy', 0.4242187738418579),
 ('cesarean', 0.42124369740486145),
 ('polydipsia', 0.4144246578216553),
 ('region', 0.40971824526786804),
 ('polyuria', 0.40000027418136597),
 ('anesthesia', 0.3928336501121521),
 ('ingestion', 0.38065752387046814),
 ('bypass', 0.3722143769264221),
 ('genitourinary', 0.37115609645843506),
 ('intestine', 0.367260217666626)]